<a href="https://colab.research.google.com/github/shinhs0920/19-lab/blob/master/Keras_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02. 심층 신경망 <br> -  ResNet & DenseNet

## 목차
* **함수형 API**
* 두 개의 입력, 한 개의 출력 **Y-Network**
* 심층 잔차 신경망(**ResNet**) - v1,v2
* 밀집 연결 합성곱 네트워크(**DenseNet**)

## 함수형 API

앞선 순차형 API를 사용한모델로는 구현할 수 없는 더 복잡한 신경망을 구축할 수 있는 도구이다.

<BR> 순차형 API와의 차이점은 각 계층은 텐서를 인수로 받으며, 모델은 입력 텐서와 출력텐서 사이의 함수다.

ex) 함수형 API에서 32개의 필터를 갖는 2차원 합성곱 계층을 Conv2D, 계층 입력텐서를 $x$, 계층 출력텐서를 $y$라 하면 아래와 같이 나타낸다.
<br> $y=Conv2D(32)(x)$ 

다음 예시는 1장에서의 MNIST코드의 CNN을 함수형 API를 사용하여 나타낸 코드이다.

In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


# MNIST dataset을 불러옵니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 레이블 개수 계산
num_labels = len(np.unique(y_train))

# 원-핫 벡터로 변환
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# 입력 이미지 reshape 및 정규화
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# 신경망 매개변수
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

# 함수형 API를 사용해 CNN 계층 구축
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,  # Convolution : 이미지에 커널, 필터를 입히는 과정 + 특징 맵(커널)의 가장자리가 삭제됨(no-padding에 의해)
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y)  # 기본적으로 pool_size=2 (2X2)를 사용하므로 인수를 제거함.
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)

# dense 계층에 연결하기 전 이미지를 알맞은 벡터로 변환
y = Flatten()(y)

# dropout 정규화
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# 입력/출력을 제공해 모델 구축
model = Model(inputs=inputs, outputs=outputs) # Model() -> 텐서 리스트를 제공해주는 메서드 

# 텍스트로 신경망 모델 요약
model.summary()

# 분류 모델 손실 함수, Adam 최적화, 정확도
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 입력 이미지와 레이블로 모델 훈련
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test), # validation_data -> 모델 훈련동안 검증 정확도의 변화 확인을 위한 인수
          epochs=20,
          batch_size=batch_size)

# 테스트 데이터세트에 대한 모델 정확도 평가
score = model.evaluate(x_test,
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0   

1장에서 순차적 API를 사용했을 때의 정확도 99.4%와 거의 같다.

##입력이 두 개, 출력이 하나인 모델 생성하기 -> Y-Network

Y-Network는 동일한 입력을 CNN의 왼쪽과 오른쪽 가지에 두번 사용한다. 이를 concatenate 계층을 사용해 결과를 결합한다.

<br>예를 들면 (3,3,16)인 두 개의 텐서를 연결하면 결과로 형상이 (3,3,32)인 텐서를 얻게 된다.

![image](https://user-images.githubusercontent.com/53015968/73603935-c93bb600-45cc-11ea-93e1-91734c10d600.png)

앞선 코드 모델의 성능을 개선하기 위해 다음 작업을 수행한다. 

1.Y-Network의 가지에서 필터 수를 두 배로 늘려 MaxPooling2D() 다음에서 특징 맵 크기가 절반으로 줄어든 것을 보완한다. 예를 들어, 첫 번째 합성곱의 출력이 (28,28,32)이면 맥스 풀링 이후에 형상이 (14,14,32)가 된다. 그러면 다음 합성곱 계층의 필터 크기는 64이고 출력은 (14,14,64)이다.


![image](https://user-images.githubusercontent.com/53015968/73604002-a3fb7780-45cd-11ea-88db-60b991a683f5.png)


2.오른쪽 한쪽 가지에서 팽창률(dilation rate)을 2로 적용한다. 팽창률을 사용해 커널의 적용 범위를 증가시키면 오른쪽 가지가 더 다양한 특징맵을 학습시킬 수 있다는 뜻이다. padding='same' 을 사용해입력 차원을 출력인 특징 맵과 동일하게 하기 위해 입력의 나머지 부분을 0으로 채워 패딩한다.

다음 코드는 함수형 API를 사용해 Y-Network를 구현하는 코드이다.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, concatenate # Y-Network의 특징인 concatenate를 사용하였다.
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model


(x_train, y_train), (x_test, y_test) = mnist.load_data()


num_labels = len(np.unique(y_train))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255


input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
dropout = 0.4
n_filters = 32

#  Y-Network의 왼쪽 가지
left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters
# Conv2D-Dropout-MaxPooling2D 3개의 층으로 구현
# 계층이 지날 때 마다 필터 개수를 2배 증가 (32-64-128)
for i in range(3):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters *= 2

# Y-Network의 오른쪽 가지
right_inputs = Input(shape=input_shape)
y = right_inputs
filters = n_filters
# Conv2D-Dropout-MaxPooling2D 3개층 구성
# 계층이 지날 때 마다 필터 개수를 2배 증가 (32-64-128)
for i in range(3):
    y = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu',
               dilation_rate=2)(y)
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    filters *= 2

# 왼쪽과 오른쪽 가지 출력 병합
y = concatenate([x, y])

# Dense 계층에 연결하기 전 특징 맵을 벡터로 변환 
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# 함수형 API에서 모델 구축
model = Model([left_inputs, right_inputs], outputs)

# 그래프를 사용해 모델 확인
plot_model(model, to_file='cnn-y-network.png', show_shapes=True)

# 계층 텍스트 설명을 사용해 모델 확인
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit([x_train, x_train],
          y_train, 
          validation_data=([x_test, x_test], y_test),
          epochs=20,
          batch_size=batch_size)

score = model.evaluate([x_test, x_test],
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 28, 28, 32)   320         input_4[0][0]                    
____________________________________________________________________________________________

또 하나의 특징은 Y-Network에서는 훈련과 검증에 두 개의 입력이 필요하므로, [x-train, x-train]이 제공된다.

아래 그림은  위 코드를 구현한 CNN MNIST Y-Network이다.

![image](https://user-images.githubusercontent.com/53015968/73604144-5849cd80-45cf-11ea-8dba-f183d6041867.png)


# 심층 잔차 신경망(ResNet)

![image](https://user-images.githubusercontent.com/53015968/73604184-cee6cb00-45cf-11ea-9869-acd291c41730.png)


심층 신경망에서 문제는 역전파가 이루어지면서 그레디언트 손실이 발생하는 것이다.
<br>역전파는 연쇄법칙을 기반으로 하며 그레디언트는 입력층으로 갈수록 손실이 진행된다.

![image](https://user-images.githubusercontent.com/53015968/73604231-677d4b00-45d0-11ea-8ef7-21f17cd90f94.png)


  ResNet 블록은 일반 CNN 블록과 비교했을 때 역전파 중 그레디언트 손실을 막기위해 **숏컷 연결**을 도입하였다.

### CNN과 ResNet 차이점

![image](https://user-images.githubusercontent.com/53015968/73604261-eecabe80-45d0-11ea-9d06-8a3f022b1280.png)

##### CNN

CNN 계층의 연산은 Conv2D - 배치 정규화- ReLU 를 통하여 진행된다. 

<br>즉 특정 계층 l의 출력 특징 맵은 이전 특징 맵에서만 직접적으로 영향받는다.

##### ResNet

ResNet 계층 연산은  Conv2D - 배치 정규화- ReLU 연산을 진행한 출력이  Conv2D - 배치 정규화 과정(잔차 매핑)의 연산을 한번 더 진행한 출력과 원래 입력값을 더하여 ReLU 함수 연산을 진행하여 출력을 낸다.

이에 대한 예제로는 CIFAR10을 활용할 것이다.
<br> CIFAR10 역시 MNIST처럼 10개의 카테고리로 되어 있다. (32X32)의 RGB로 되어있다. 5만개의 레이블이 달린 훈련 이미지와, 1만개의 검증 이미지로 구성되어 있다. 

아래 그림은 CIFAR10 데이터세트에 대한 ResNet 모델 아키텍처를 보여준다.

![image](https://user-images.githubusercontent.com/53015968/73604362-eecbbe00-45d2-11ea-9d0b-cd615276b38a.png)

아래 표는 ResNet 아키텍쳐 구성이다.
![image](https://user-images.githubusercontent.com/53015968/73604469-b5944d80-45d4-11ea-8301-6d74a02f3b30.png)


이는 잔차 블록 집합이 3개 있다는 뜻이며, 각 집합에는 n개의 잔차 블록이 있다면 2n개의 계층이 있다.

<br> 서로 다른 크기의 두 특징 맵 사이의 전이를 제외하면 커널 크기는 전부 3 이다.전이 계층에서는 커널 크기가 1이고 strides=2 일때의 Conv2D이다.

*) strides=2 뜻은 합성곱 적용 시, 픽셀을 하나씩 건너뛰면서 커널을 적용

<BR> 서로 다른 크기의 두 잔차 블록을 연결할 때 전이 계층을 사용한다.

마지막 계층은 AveragePooling2D-Faltten-Dense로 구성된다. ResNet은 dropout을 사용하지 않는다. 또한, 덧셈 연산과 1X1 합성곱 적용은 자신을 정규화하는 효과를 가진다.

CIFAR10 데이터세트 분류를 위한 ResNet 아키텍쳐
![image](https://user-images.githubusercontent.com/53015968/73604527-b1b4fb00-45d5-11ea-99c5-b54ee8b50375.png)

In [0]:
# ResNet v1과 ResNet v2가 모두 구현되어 있다.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# 훈련 매개변수
batch_size = 32
epochs = 200
data_augmentation = True
num_classes = 10

subtract_pixel_mean = True

# 모델 파라미터
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# 모델 버전
# 최초 논문: version = 1 (ResNet v1), 
version = 1

# 제공된 모델 매개변수 n으로부터 계산된 네트워크 깊이
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

model_type = 'ResNet%dv%d' % (depth, version)

# CIFAR10 데이터 불러오기
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# input image dimensions.
input_shape = x_train.shape[1:]

# 정규화
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """ 학습률은 lr_schedule()을 통해 80, 120, 160, 180 epochs 에서 시작하여 점차 감소시킨다.
    기본값은 1e-3 이다. lr_schedule()함수는 모델 훈련동안 한 epoch 당 callbacks 변수로 호출된다.

    # Arguments
        에폭의 수 :-> epoch (int)
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    Arguments:
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    Returns:
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal', # 역전파 시 수렴될 수 있게 해주는 것
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')
    
    # 모델 정의 시작
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    
    # 잔차 유닛 인스턴스화
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
           
            if stack > 0 and res_block == 0:  
                strides = 2  
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            
            if stack > 0 and res_block == 0:
               
               # 변경된 차원을 맞추기 위해 잔차 숏컷 연결을 선형으로 사영 
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # 분류기 추가
    # v1은 마지막 숏컷 연결-ReLU 후에는 BN을 사용하지 않는다.
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # 모델 인스턴스화
    model = Model(inputs=inputs, outputs=outputs)
    return model

#ResNet v2 구현

def resnet_v2(input_shape, depth, num_classes=10):
   
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 110 in [b])')
   
   # 모델 정의
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
   # v2에서는 2 경로로 나뉘기 전에 입력에 BN-ReLU와 함께 Conv2D를 수행 
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                # first layer and first stage
                if res_block == 0:  
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
               
                if res_block == 0:
                   
                    strides = 2 

           
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
              
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])

        num_filters_in = num_filters_out

    # add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
plot_model(model, to_file="%s.png" % model_type, show_shapes=True)
print(model_type)

# prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # this will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, 
                        steps_per_epoch=len(x_train)//batch_size,
                        callbacks=callbacks)

# score trained model
scores = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
_______________________

## ResNet v2 - 개선된 ResNet 모델

### ResNet v1 과 ResNet v2 비교

![image](https://user-images.githubusercontent.com/53015968/73605043-8f72ab80-45dc-11ea-8a66-530581ea7d76.png)


* 연산량을 줄이기 위해 1X1-3X3-1X1 BN=ReLU-Conv2D 스택을 사용 - 병목 레이어 (bottleneck layer)
* 배치 정규화와 ReLU 활성화 함수를 2D 합성곱 전에 위치

## 밀집 연결 합성곱 네트워크(DenseNet)

Dense는 ResNet과 달리 숏컷 연결이 아닌 이전 특징 맵 전체가 다음 계층의 입력이 되는 방식으로 그레디언트 소실문제를 해결한다.

![image](https://user-images.githubusercontent.com/53015968/73605264-833c1d80-45df-11ea-976d-d6d88cb731fd.png)


Conv2D 역시 크기가 3인 커널을 사용한다.

계층마다 생성되는 특징 맵의 개수를 성장률$k$라고 한다.
<br>일반적으로 $k$=12를 사용한다. 특정 맵 $x_0$ 개수가 $k_0$일 때 위 그림처럼 4개의 계층으로 구성된 밀집 계층의 끝에서 특징 맵의 전체 개수는 $4 \times k +k_0 $이다. 

![image](https://user-images.githubusercontent.com/53015968/73605405-2b9eb180-45e1-11ea-9d25-a247de91a561.png)

병합 연산에서 연결(Concat)을 사용하기 위해 크기의 차이를 일치하여야 한다. 특징 맵 개수가 효율적인 개수까지 증가하도록 하기위해 우측 그림처럼 병목 계층을 도입했다. 

연결 후에는 필터 크기가 $4k$인 $1 \times 1$ 합성곱이 적용된다.
<br> 이 것은 Conv2D(3)에서 처리되는 특징 맵 개수가 빠르게 증가하는 것을 막는다.

<br> 병목 도입을 안할 떈 Conv2D(3)에서 받는 입력은 1224개고
병목계층을 도입한 후에는 48개의 특징 맵만 입력받는다.

![image](https://user-images.githubusercontent.com/53015968/73605482-33128a80-45e2-11ea-9290-261585335e8e.png)


특징 맵 크기는 서로 다 다르기 때문에 이를 해결하기 위해 DenseNet은 심층 신경망을 여러 개의 밀집 블록으로 나누고 전이 계층을 통해 서로 연결시킨다. 

각 밀집 블록 내에서는 특징 맵 크기(너비, 높이)가 일정하다.

전이 계층 : 두 밀집 블록 사이에서 한 특징 맵 크기에서 더 작은 특징 맵 크기로 바꾸는 것 (일반적으론 절반으로 줄인다)

<br> 위 기능은 AveragePooling2D 계층이 수행한다.

예를 들면, 기본 pool_size=2 를 적용한 AveragePooling2D를 사용하면 맵 크기가 (64,64,256)에서 (32,32,256)으로 줄어든다.

하지만, 특징 맵이 AveragePooling2D 으로 전달되기 전에 Conv2D(1)을 사용해 맵 개수를 특정 압축비 $0<\theta<1$로 줄인다. 여기선 0.5로 정한다.

따라서 (64,64,512)의 출력이 Conv2D에서 (64,64,256)으로 줄어들고, AveragePooling2D를 통해 (32,32,256)으로 한번더 줄어드는 것이다.

### CIFAR10을 위한 100계층 DenseNet-BC 구성하기

이제 CIFAR10 데이터 세트에 100개의 계층으로 구성된 DenseNet-BC(병목 압축)을 구성한다.

아래 표는 100개의 계층으로 이루어진 DenseNet-BC이다.

![image](https://user-images.githubusercontent.com/53015968/73605581-6275c700-45e3-11ea-8ba9-256081c85a5c.png)


![image](https://user-images.githubusercontent.com/53015968/73605596-93ee9280-45e3-11ea-8c40-9d071d9cf6fb.png)


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np

# 훈련 매개변수
batch_size = 32
epochs = 200
data_augmentation = True

# 네트워크 매개변수
num_classes = 10
num_dense_blocks = 3
use_max_pool = False

# DenseNet-BC 데이터 셋
# Growth rate   | Depth |  Accuracy (paper)| Accuracy (this)      |
# 12            | 100   |  95.49%          | 93.74%               |
# 24            | 250   |  96.38%          | requires big mem GPU |
# 40            | 190   |  96.54%          | requires big mem GPU |
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)

num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

# CIFAR10 데이터 불러오기
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


input_shape = x_train.shape[1:]

# 데이터 정규화
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


# 모델 정의
# densenet CNNs (합성함수)는 BN-ReLU-Conv2D로 구성됨
inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,
           kernel_size=3,
           padding='same',
           kernel_initializer='he_normal')(x)
x = concatenate([inputs, x])

# 전이층으로 연결된 밀집 블록의 스택
for i in range(num_dense_blocks):
    # 밀집 블록은 병목 계층의 스택이다
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,
                   kernel_size=1,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        x = concatenate([x, y])

    # 마미작 밀집 블록 다음에는 전이 계층이 없다
    if i == num_dense_blocks - 1:
        continue

    # 전이 계층에서 특징 맵 개수를 압축하고 크기를 1/2로 줄인다.
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,
               kernel_size=1,
               padding='same',
               kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)


# 애버리지 풀링 다음으로 상단에 분류 모델 추가
# 특징 맵 크기는1 x 1
x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                kernel_initializer='he_normal',
                activation='softmax')(y)

# 최초 논문에서는 SGD를 사용했지만 DenseNet에서는 RMSprop 가 성능이 좋다.
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(1e-3),
              metrics=['accuracy'])
model.summary()
plot_model(model, to_file="cifar10-densenet.png", show_shapes=True)

# prepare model model saving directory
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate reducer
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# run training, with or without data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # preprocessing  and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(x_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1,
                        callbacks=callbacks)

# score trained model
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])